<a href="https://colab.research.google.com/github/karimfathy054/Clustering/blob/main/Third_clustering_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Note:

the code is tested on small data from sheets it needs further testing

# Loading dataset

In [ ]:
!unzip /content/drive/MyDrive/activities.zip -d /home

In [2]:
dataset_path = "/home/data"

In [3]:
import numpy as np
import pandas as pd
import os
from natsort import natsorted
from sklearn.decomposition import PCA

In [5]:
def load_data(data_path,reduction_method="mean"):
  training_data=[]
  training_labels=[]
  evaluation_data=[]
  evaluation_labels=[]
  for i,activity in enumerate(natsorted(os.listdir(data_path))):
    activity_path = os.path.join(data_path,activity)
    for subject in natsorted(os.listdir(activity_path)):
      subject_path = os.path.join(activity_path,subject)
      data = []
      for j,segment in enumerate(natsorted(os.listdir(subject_path))):
        segment_path = os.path.join(subject_path,segment)
        data_point = pd.read_csv(segment_path,header=None).values
        if(reduction_method == "mean"):
          data.append(np.mean(data_point,axis=0))
        elif(reduction_method == "pca"):
          data.append(data_point.flatten())
      if(reduction_method == "pca"):
        data = np.vstack(data)
        data = PCA(n_components=45).fit_transform(data)
        data = data.tolist()
      training_data += data[:48]
      training_labels += [i]*48
      evaluation_data += data[48:]
      evaluation_labels += [i]*12
  return training_data,training_labels,evaluation_data,evaluation_labels

In [6]:
training_data,training_labels,evaluation_data,evaluation_labels = load_data(dataset_path)

In [ ]:
training_data,training_labels,evaluation_data,evaluation_labels = load_data(dataset_path,reduction_method="pca")

# trying sklearn's algorithm

In [50]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import rand_score
from sklearn.metrics.cluster import homogeneity_completeness_v_measure

In [54]:
hc = AgglomerativeClustering(n_clusters = 19,linkage="ward")
predictions = hc.fit_predict(training_data)
print(rand_score(training_labels,predictions))
print(adjusted_rand_score(training_labels,predictions))
purity , recall , v_measure = homogeneity_completeness_v_measure(training_labels,predictions)
print(purity)
print(recall)
print(v_measure)

0.8807571801762804
0.2340834704241038
0.5137782422513342
0.6085773063531851
0.5571742022765497


In [48]:
dbsc = DBSCAN(eps=2,min_samples=5)
predictions = dbsc.fit_predict(training_data)
adjusted_rand_score(training_labels,predictions)

0.21294252791092105

In [37]:
predictions

array([   0,    1,    2, ..., 6247, 6248, 6249])

# Expectation Maximization Clustering Algorithm

expectation maximization is classified as _soft assignment clustering_ algorithm that differs _hard assignment clustering algorithms_ like **k-means**:

a data sample can be in multiple clusters at the same time with different probabilities



In [ ]:
from scipy.stats import multivariate_normal

In [ ]:
def expectationMaximization(data,k,max_itr=10000,e=1e-6):
    n_samples , n_features = data.shape;

    # choose random k from data to act as our random initial means
    means = np.array(data[np.random.choice(n_samples,k,replace=False)],dtype=float)

    prior_probs = np.ones(k)/k #initial prior probabilities for each cluster
    covariances = np.array([np.eye(n_features) for i in range(k)]) # using diagonal covariance matrix for simplicity
    weights = np.zeros((n_samples,k))
    for itr in range(max_itr):
        # Expectation
        # calculating probability that each sample is in a given class
        for i in range(n_samples):
            for j in range(k):
                weights[i,j] = prior_probs[j]*multivariate_normal(mean = means[j],cov = covariances[j],allow_singular=True).pdf(data[i])
            weights[i] = weights[i]/np.sum(weights[i])+1e-6


        # Maximization step
        old_means = means.copy()
        for i in range(k):
            means[i] = np.dot(data.T,weights[:,i])/np.sum(weights[:,i]) # calculating new mean for each class based on the probability for all the points to be belonging to it
            covariances[i] = np.sum(
                np.array([weights[j,i]*np.outer(data[j]-means[i],data[j]-means[i]) for j in range(n_samples)]),axis=0
                )/np.sum(weights[:,i]) # calculating new covariance matrices using the new calculated means
            prior_probs[i] = np.sum(weights[:,i])/n_samples

        if np.all(means-old_means)<e:
            break
    predictions = np.argmax(weights,axis=1)
    return predictions,means,covariances;




In [ ]:
predictions,means,covariances = expectationMaximization(np.vstack(training_data),k=19)
predictions

array([9, 9, 9, ..., 3, 3, 3])

In [ ]:
from sklearn.metrics import rand_score
rand_score(training_labels,predictions)

0.9135442218895422

## Hierarchical clustering

not running yet

In [ ]:
from sklearn.metrics import pairwise_distances

In [ ]:
def cluster_distance(cluster1, cluster2, distance_between_clusters="single_link"):
    distance = 0.0;
    if(distance_between_clusters == "single_link"):
        distance = np.min(pairwise_distances(cluster1,cluster2))
    elif(distance_between_clusters == "complete_link"):
        distance = np.max(pairwise_distances(cluster1,cluster2))
    elif(distance_between_clusters == "group_average"):
        distance = np.mean(pairwise_distances(cluster1,cluster2))
    elif(distance_between_clusters == "mean_distance"):
        mean1= np.mean(np.vstack(cluster1),axis=0)
        mean2= np.mean(np.vstack(cluster2),axis=0)
        distance = np.linalg.norm(mean1-mean2)
    elif(distance_between_clusters == "wards"):
        mean1= np.mean(np.vstack(cluster1),axis=0)
        mean2= np.mean(np.vstack(cluster2),axis=0)
        distance = (len(cluster1)*len(cluster2)/len(cluster1)+len(cluster2))*np.linalg.norm(mean1-mean2)**2
    return distance

In [ ]:
def hierarchical_clustering(data,k,distance_between_clusters = "single_link"):
    n_samples = data.shape[0]
    n_clusters = n_samples
    distances = pairwise_distances(data)
    clusters=[]
    for point in data:
        clusters.append([point])
    for i in range(n_samples):
        distances[i,i] = np.inf
    while n_clusters > k:
        min_indx1,min_indx2 = np.unravel_index(np.argmin(distances),distances.shape)
        culster1 = clusters[min_indx1]
        culster2 = clusters[min_indx2]
        Cij = culster1 + culster2

        for i in range(n_clusters):
            if(i == min_indx1 or i == min_indx2):
                continue
            dCiCr = cluster_distance(clusters[min_indx1],clusters[i],distance_between_clusters)
            dCjCr = cluster_distance(clusters[min_indx2],clusters[i],distance_between_clusters)
            dCiCj = cluster_distance(clusters[min_indx1],clusters[min_indx2],distance_between_clusters)
            alpha_i = 0.0
            alpha_j = 0.0
            beta = 0.0
            gamma = 0.0
            if(distance_between_clusters == "single_link"):
                alpha_i = 0.5
                alpha_j = 0.5
                beta = 0
                gamma = -0.5
            elif(distance_between_clusters == "complete_link"):
                alpha_i = 0.5
                alpha_j = 0.5
                beta = 0
                gamma = 0.5
            elif(distance_between_clusters == "group_average"):
                alpha_i = len(clusters[min_indx1])/(len(clusters[min_indx1])+len(clusters[min_indx2]))
                alpha_j = len(clusters[min_indx2])/(len(clusters[min_indx1])+len(clusters[min_indx2]))
                beta = 0
                gamma = 0
            elif(distance_between_clusters == "mean_distance"):
                alpha_i = len(clusters[min_indx1])/(len(clusters[min_indx1])+len(clusters[min_indx2]))
                alpha_j = len(clusters[min_indx2])/(len(clusters[min_indx1])+len(clusters[min_indx2]))
                beta = -len(clusters[min_indx1])*len(clusters[min_indx2])/(len(clusters[min_indx1])+len(clusters[min_indx2]))**2
                gamma = 0
            elif(distance_between_clusters == "wards"):
                alpha_i = len(clusters[min_indx1])+len(clusters[i])/len(clusters[min_indx1])+len(clusters[min_indx2])+len(clusters[i])
                alpha_j = len(clusters[min_indx2])+len(clusters[i])/len(clusters[min_indx1])+len(clusters[min_indx2])+len(clusters[i])
                beta  = -len(clusters[i])/len(clusters[min_indx1])+len(clusters[min_indx2])+len(clusters[i])
                gamma = 0
            distances[min_indx1,i]=distances[i,min_indx1] = alpha_i*dCiCr + alpha_j*dCjCr + beta*dCiCj + gamma*np.abs(dCiCr-dCjCr)
        clusters[min_indx1] = Cij
        clusters.pop(min_indx2)
        distances = np.delete(distances,min_indx2,axis=0)
        distances = np.delete(distances,min_indx2,axis=1)
        n_clusters -= 1
    return clusters,distances